In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from PIL import Image
import os


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # centre les valeurs around 0
])

train_dir = 'stegoimagesdataset/train/train/'
val_dir = 'stegoimagesdataset/val/val/'
test_dir = 'stegoimagesdataset/test/test/'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Classes: {train_dataset.classes}")
number_of_classes = len(train_dataset.classes)


Classes: ['clean', 'stego']


In [34]:
import torch
import torch.nn as nn
import torchvision.models as models

# Use the new weights parameter instead of pretrained
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)


# Replace the last layer to match the number of classes in the dataset
model.fc = nn.Linear(model.fc.in_features, number_of_classes)  # 2 classes : clean and stego

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch.optim as optim

def train(model, train_loader, criterion, optimizer, num_epochs=10):

    for epoch in range(num_epochs):
        model.train() # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs in train_loader:
            #inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()


            outputs = model(inputs[0]) # Forward images through the model (Forward pass)
            loss = criterion(outputs, inputs[1]) # Calculate loss, moyenne de la perte sur le batch actuel,(perte 1 + perte 2 + ... + perte n) / n
            loss.backward() # Backpropagation of the loss
            optimizer.step() # Update weights

            running_loss += loss.item() * len(inputs[0]) # Donne la somme totale de la perte sur le batch
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == inputs[1]).sum().item()
            total += len(inputs[1])

        epoch_loss = running_loss / total # Average loss over the entire dataset
        accuracy = 100 * correct / total # Calculate accuracy
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return model

criterion = nn.CrossEntropyLoss() # Cross entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=1e-4) # Adam optimizer

trained_model = train(model, train_loader, criterion, optimizer)

Epoch [1/10], Loss: 0.4822, Accuracy: 72.94%
Epoch [2/10], Loss: 0.4295, Accuracy: 74.76%
Epoch [3/10], Loss: 0.4220, Accuracy: 74.72%
Epoch [4/10], Loss: 0.4168, Accuracy: 74.68%
Epoch [5/10], Loss: 0.4109, Accuracy: 74.63%


In [ ]:
# validation step
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return running_loss / len(val_loader), accuracy

